# Retrieval Augumented Generation (RAG) inference

***This notebook works best with the `conda_python3` on the `ml.t3.large` instance***.

---

At this point our slide deck data is ingested into Amazon OpenSearch Service Serverless collection. We are now ready to talk to our slide deck using a large multimodal model. We are using the [Anthropic’s Claude 3 Sonnet foundation model](https://aws.amazon.com/about-aws/whats-new/2024/03/anthropics-claude-3-sonnet-model-amazon-bedrock/) for this purpose.

## Step 1. Setup

Install the required Python packages and import the relevant files.

In [40]:
import sys
!{sys.executable} -m pip install -r requirements.txt

  Cloning https://github.com/haotian-liu/LLaVA.git (to revision v1.1.1) to /tmp/pip-req-build-lj64bofx
  Running command git clone --filter=blob:none --quiet https://github.com/haotian-liu/LLaVA.git /tmp/pip-req-build-lj64bofx
  Running command git checkout -q 1619889c712e347be1cb4f78ec66e7cf414ac1a6
  Resolved https://github.com/haotian-liu/LLaVA.git to commit 1619889c712e347be1cb4f78ec66e7cf414ac1a6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [55]:
import os
import io
import sys
import json
import glob
import boto3
import base64
import logging
import requests
import botocore
import sagemaker
import numpy as np
import pandas as pd
import globals as g
from PIL import Image
from pathlib import Path
from typing import List, Dict
from IPython.display import Image
from urllib.parse import urlparse
from botocore.auth import SigV4Auth
from pandas.core.series import Series
from sagemaker import get_execution_role
from botocore.awsrequest import AWSRequest
from utils import get_cfn_outputs, get_text_embedding, get_llm_response
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth


In [56]:
!pygmentize globals.py

"""
Global variables used throughout the code.
"""
import os
import boto3
import sagemaker

# S3 bucket strucutre, we use the default sagemaker bucket in the current region
# BUCKET_NAME: str = sagemaker.Session().default_bucket()
BUCKET_PREFIX: str = "multimodal"
BUCKET_EMB_PREFIX: str = f"{BUCKET_PREFIX}/osi-embeddings-json"
BUCKET_IMG_PREFIX: str = f"{BUCKET_PREFIX}/img"
LLAVA_PROMPT_PREFIX: str = 'multimodal/llavaPrompt/'

S3_MODEL_CODE_PREFIX: str = "code"
S3_MODEL_PREFIX: str = "model"

# Amazon Titan multimodal model
AWS_REGION: str = boto3.Session().region_name
FMC_URL: str = f"https://bedrock-runtime.{AWS_REGION}.amazonaws.com"
FMC_MODEL_ID: str = "amazon.titan-embed-text-v1"
CLAUDE_MODEL_ID: str = "anthropic.claude-3-haiku-20240307-v1:0"
ACCEPT_ENCODING: str = "application/json"
CONTENT_ENCODING: str = "application/json"

# Amazon OpenSearch Service Serverless
OS_SERVICE: str = "aoss"

# local files and folder structure
IMAGE_DIR: str = "img"
IMAGE_FILE_EXTN: str = ".jpg"
B64

In [57]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

## Step 2. Create an OpenSearch client and SageMaker Predictor object

We create an OpenSearch client so that we can query the vector database for embeddings (slides) similar to the questions that we might want to ask of our slide deck and then we create a SageMaker [`Predictor`](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html) to run inference using the LLaVA model given the slide we retrieved from OpenSearch.

Get the name of the OpenSearch Service Serverless collection endpoint and index name from the CloudFormation stack outputs.

In [58]:
outputs = get_cfn_outputs(g.CFN_STACK_NAME)
host = outputs['MultimodalCollectionEndpoint'].split('//')[1]
index_name = outputs['OpenSearchIndexName']
logger.info(f"opensearchhost={host}, index={index_name}")


[2024-03-18 22:11:06,771] p29487 {2998865555.py:5} INFO - opensearchhost=erl0gs7pm3an9js60q6j.us-east-1.aoss.amazonaws.com, index=attempt2


In [70]:
session = boto3.Session()
credentials = session.get_credentials()
auth = AWSV4SignerAuth(credentials, g.AWS_REGION, g.OS_SERVICE)

os_client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

[2024-03-18 22:16:58,420] p29487 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


## Step 3. Read for RAG

We now have all the pieces for RAG. Here is how we _talk to our slide deck_.

1. Convert the user question into embeddings using the Titan Text Embeddings model.

1. Find the most similar slide (image) corresponding to the the embeddings (for the user question) from the vector database (OpenSearch Serverless).

1. Now ask Claude3 to answer the user question using the retrieved image description for the most similar slide.

In [71]:
bedrock = boto3.client(service_name="bedrock-runtime", endpoint_url=g.FMC_URL)

A handy function for similarity search in the vector db

In [72]:
def find_similar_data(text_embeddings: np.ndarray) -> Dict:
    query = {
        "size": 1,
        "query": {
            "knn": {
                "vector_embedding": {
                    "vector": text_embeddings,
                    "k": 1
                }
            }
        }
    }
    try:
        image_based_search_response = os_client.search(body=query, index=index_name)
        # remove the vector_embedding field for readability purposes, it was needed during
        # the similarity search (by the vector db), we do not need it any more.
        source = image_based_search_response['hits']['hits'][0]['_source'].pop('vector_embedding')
        logger.info(f"received response from OpenSearch, response={json.dumps(image_based_search_response, indent=2)}")
    except Exception as e:
        logger.error(f"error occured while querying OpenSearch index={index_name}, exception={e}")
        image_based_search_response = None
    return image_based_search_response

### Question 1

Create a prompt and convert it to embeddings.

In [73]:
question: str = "How does Inf2 compare in performance to comparable EC2 instances? I need numbers."
text_embedding = get_text_embedding(bedrock, question)

Find the most similar slide from the vector db.

In [74]:
vector_db_response: Dict = find_similar_data(text_embedding)

[2024-03-18 22:17:02,679] p29487 {base.py:259} INFO - POST https://erl0gs7pm3an9js60q6j.us-east-1.aoss.amazonaws.com:443/attempt2/_search [status:200 request:0.113s]
[2024-03-18 22:17:02,680] p29487 {1697326408.py:20} ERROR - error occured while querying OpenSearch index=attempt2, exception=list index out of range


Retrieve the image path from the search results and provide it to Claude3 along with the user question.

In [75]:
s3_img_path = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('image_path')
logger.info(f"going to answer the question=\"{question}\" using the image \"{s3_img_path}\"")

!aws s3 cp {s3_img_path} .
local_img_path = os.path.basename(s3_img_path)
Image(filename=local_img_path) 

AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
slide_text = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('slide_text')
print(slide_text)

In [ ]:
llm_response = get_llm_response(bedrock, question, slide_text)
print(llm_response)

### Question 2

In [51]:
question: str = "As per the AI/ML flywheel, what do the AWS AI/ML services provide?"
text_embedding = get_text_embedding(bedrock, question)
vector_db_response: Dict = find_similar_data(text_embedding)
s3_img_path = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('image_path')
logger.info(f"going to answer the question=\"{question}\" using the image \"{s3_img_path}\"")

!aws s3 cp {s3_img_path} .
local_img_path = os.path.basename(s3_img_path)
Image(filename=local_img_path) 
slide_text = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('slide_text')
print(slide_text)
llm_response = get_llm_response(bedrock, question, slide_text)
print(llm_response)


[2024-03-18 22:09:17,568] p29487 {base.py:259} INFO - POST https://erl0gs7pm3an9js60q6j.us-east-1.aoss.amazonaws.com:443/attempt2/_search [status:200 request:0.114s]
[2024-03-18 22:09:17,570] p29487 {1697326408.py:20} ERROR - error occured while querying OpenSearch index=attempt2, exception=list index out of range


AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
# create prompt and convert to embeddings
question: str = "As per the AI/ML flywheel, what do the AWS AI/ML services provide?"
text_embedding = get_text_embedding(bedrock, question)

# vector db search
vector_db_response: Dict = find_similar_data(text_embedding)

# download image for local notebook display
s3_img_path = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('image_path')
logger.info(f"going to answer the question=\"{question}\" using the image \"{s3_img_path}\"")

!aws s3 cp {s3_img_path} .
local_img_path = os.path.basename(s3_img_path)
display(Image(filename=local_img_path))

# Ask Claude
slide_text = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('slide_text')
print(slide_text)
llm_response = get_llm_response(bedrock, question, slide_text)
print(llm_response)

### Question 3

What about slides that contain charts and graphs? We want to see if the LLaVA model can correcly analyze a graph and pull appropriate metrics from the slide. 

In [ ]:
# create prompt and convert to embeddings
question: str = "Compared to GPT-2, how many more parameters does GPT-3 have? What is the numerical difference between the parameter size of GPT-2 and GPT-3?"
text_embedding = get_text_embedding(bedrock, question)

# vector db search
vector_db_response: Dict = find_similar_data(text_embedding)

# download image for local notebook display
s3_img_path = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('image_path')
logger.info(f"going to answer the question=\"{question}\" using the image \"{s3_img_path}\"")

!aws s3 cp {s3_img_path} .
local_img_path = os.path.basename(s3_img_path)
display(Image(filename=local_img_path))

# Ask Claude
slide_text = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('slide_text')
print(slide_text)
llm_response = get_llm_response(bedrock, question, slide_text)
print(llm_response)

### Question 4

How about a question that cannot be answered based on this slide deck? We want to confirm that while some slide image will be retrieved but the Claude model does not hallucinate and correctly says  "I do not know".

In [ ]:
# create prompt and convert to embeddings
question: str = "What are quarks in particle physics?"
text_embedding = get_text_embedding(bedrock, question)

# vector db search
vector_db_response: Dict = find_similar_data(text_embedding)

# download image for local notebook display
s3_img_path = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('image_path')
logger.info(f"going to answer the question=\"{question}\" using the image \"{s3_img_path}\"")

!aws s3 cp {s3_img_path} .
local_img_path = os.path.basename(s3_img_path)
display(Image(filename=local_img_path))

# Ask Claude
slide_text = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('slide_text')
print(slide_text)
llm_response = get_llm_response(bedrock, question, slide_text)
print(llm_response)

## Clean Up
